In [ ]:
%pip install --upgrade pip
%pip install selenium chromedriver-autoinstaller schedule
!apt-get update
!apt-get install -y chromium-browser chromium-chromedriver


In [ ]:
import requests

url = "https://googlechromelabs.github.io/chrome-for-testing/latest-versions-per-milestone-with-downloads.json"
response = requests.get(url, verify=False)  # Disable SSL verification

print(response.json())


In [ ]:
%pip install --upgrade pip
%pip install selenium chromedriver-autoinstaller schedule


In [ ]:
import time
import random
import os
import schedule
import smtplib
from datetime import datetime, timedelta
from collections import defaultdict
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller

#############################
# CONFIGURATION
#############################

HEADLESS = True  
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
SENDER_EMAIL = "shivamjoshi70@gmail.com"  
SENDER_PASSWORD = "fnctuagbboiitzgu"   
RECIPIENTS = ["shivamjoshi70@gmail.com"]

OUTPUT_FILENAME = "tenders_output.txt"

#############################
# FUNCTION: Setup WebDriver
#############################

def setup_driver():
    """Setup Chrome WebDriver with options."""
    chromedriver_autoinstaller.install()
    chrome_options = Options()
    if HEADLESS:
        chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

#############################
# FUNCTION: Web Scraping
#############################

def search_and_scrape(keywords, from_date, to_date, max_pages=3):
    driver = setup_driver()
    result_links = defaultdict(list)

    try:
        for keyword in keywords:
            print(f"\n🔍 Searching for: {keyword}")
            driver.get("https://www.find-tender.service.gov.uk/Search/Results")
            time.sleep(random.uniform(2, 3))

            search_box = WebDriverWait(driver, 8).until(
                EC.visibility_of_element_located((By.ID, "keywords"))
            )
            search_box.clear()
            search_box.send_keys(keyword)
            time.sleep(random.uniform(1, 2))
            search_box.send_keys(Keys.RETURN)
            time.sleep(random.uniform(1.5, 2.5))

            all_links = set()
            for page in range(1, max_pages + 1):
                paginated_url = driver.current_url + f"&page={page}#dashboard_notices"
                print(f"\n📄 Fetching page {page}: {paginated_url}")
                driver.get(paginated_url)
                time.sleep(random.uniform(2, 3))

                try:
                    tender_elements = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, '/Notice/')]"))
                    )
                except:
                    print(f"⏳ No more results on page {page}.")
                    break

                page_links = {elem.get_attribute("href") for elem in tender_elements if elem.get_attribute("href")}
                print(f"✅ Found {len(page_links)} tenders on page {page}.")
                all_links.update(page_links)

            result_links[keyword] = list(all_links)
    finally:
        driver.quit()
    return result_links

#############################
# FUNCTION: Format & Save Results
#############################

def save_results_to_file(results, filename=OUTPUT_FILENAME):
    with open(filename, "w", encoding="utf-8") as f:
        for keyword, links in results.items():
            f.write(f"\n🔍 {keyword} - {len(links)} results:\n")
            f.write("\n".join(links) + "\n")
    print(f"💾 Results saved: {filename}")

#############################
# FUNCTION: Send Email
#############################

def send_email(subject, body, attachment_path):
    msg = MIMEMultipart()
    msg["From"] = SENDER_EMAIL
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    with open(attachment_path, "rb") as attachment:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header("Content-Disposition", f"attachment; filename={attachment_path}")
    msg.attach(part)

    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(SENDER_EMAIL, SENDER_PASSWORD)
        for recipient in RECIPIENTS:
            msg["To"] = recipient
            server.sendmail(SENDER_EMAIL, recipient, msg.as_string())
        server.quit()
        print("✅ Email sent successfully.")
    except Exception as e:
        print(f"❌ Email error: {e}")

#############################
# MAIN FUNCTION
#############################

def main():
    keywords = ['"THAMES WATER UTILITIES LIMITED"', '"SOUTHERN WATER UTILITIES LIMITED"',
                '"SOUTH WEST WATER LIMITED"', '"South West Water Ltd"', '"PENNON GROUP PLC"',
                '"SSE Plc"', '"SSE SERVICES PLC"', '"SEVERN TRENT WATER LIMITED"',
                '"NORTHUMBRIAN WATER GROUP LIMITED"', '"Portsmouth Water Limited"',
                '"AFFINITY WATER LIMITED"', '"Dwr Cymru Welsh Water"',
                '"UK POWER NETWORKS (OPERATIONS) LIMITED"', '"CENTRICA PLC"',
                '"GNI (UK) Limited"', '"UNITED UTILITIES WATER LIMITED"', '"ANGLIAN WATER SERVICES LIMITED"']

    today = datetime.now()
    from_date = today - timedelta(days=1)
    to_date = today

    max_pages = 1  

    start_time = time.time()
    results = search_and_scrape(keywords, from_date, to_date, max_pages=max_pages)
    save_results_to_file(results)

    execution_time = time.time() - start_time
    print(f"🚀 Execution completed in {execution_time:.2f} seconds")

    email_subject = f"Tender Alert - {today.strftime('%Y-%m-%d')}"
    email_body = "Please find attached the daily tender results."
    send_email(email_subject, email_body, OUTPUT_FILENAME)

if __name__ == "__main__":
    main()
